In [1]:
import sys
import os

sys.path.append("../")
print(os.getcwd())

D:\Dokumenty\soukromé dokumenty\VŠE\diplomka\github\rulesRandomForest\randomForestRules\notebooks


In [2]:
%run ../main.py

In [3]:
from randomForest import RandomForest
import pandas as pd

In [4]:
df = pd.read_csv("data/audiology.csv")
df.columns = df.columns.str.replace("_", "-") # underscore not allowed
df.head()

,age-gt-60,air,airBoneGap,ar-c,ar-u,bone,boneAbnormal,bser,history-buzzing,history-dizziness,...,s-sn-gt-1k,s-sn-gt-2k,s-sn-gt-4k,speech,static-normal,tymp,viith-nerve-signs,wave-V-delayed,waveform-ItoV-prolonged,binaryClass
0,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,P
1,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,P
2,t,normal,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,N
3,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,unmeasured,t,a,f,f,f,N
4,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,normal,t,a,f,f,f,P


In [5]:
df['target'] = df['binaryClass'].apply(lambda x: 1 if x == "P" else 0)
df.head()

,age-gt-60,air,airBoneGap,ar-c,ar-u,bone,boneAbnormal,bser,history-buzzing,history-dizziness,...,s-sn-gt-2k,s-sn-gt-4k,speech,static-normal,tymp,viith-nerve-signs,wave-V-delayed,waveform-ItoV-prolonged,binaryClass,target
0,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,good,t,a,f,f,f,P,1
1,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,good,t,a,f,f,f,P,1
2,t,normal,f,normal,normal,unmeasured,f,?,f,f,...,f,f,good,t,a,f,f,f,N,0
3,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,unmeasured,t,a,f,f,f,N,0
4,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,normal,t,a,f,f,f,P,1


In [6]:
cols=[]
for col in df.columns:
    if col != 'binaryClass' and col != 'target':
        cols.append(col)

In [7]:
randomForest = RandomForest()
randomForest.load_pandas(df)
randomForest.fit(antecedents = cols, consequent = 'target', supp=0.005, conf=50)
frame = randomForest.get_frame()

In [8]:
print(frame)

    age-gt-60       air airBoneGap ar-c      ar-u        bone boneAbnormal  \
0           f       NaN        NaN  NaN       NaN         NaN          NaN   
1           t       NaN        NaN  NaN       NaN         NaN          NaN   
2           t       NaN        NaN  NaN       NaN         NaN          NaN   
3           t       NaN        NaN  NaN       NaN         NaN          NaN   
4           t       NaN        NaN  NaN       NaN         NaN          NaN   
5           t       NaN        NaN  NaN       NaN      normal          NaN   
6           t       NaN        NaN  NaN       NaN  unmeasured          NaN   
7           t       NaN        NaN  NaN       NaN        mild          NaN   
8           t       NaN        NaN  NaN       NaN    moderate          NaN   
9           t       NaN        NaN  NaN       NaN           ?          NaN   
10          t       NaN        NaN  NaN       NaN         NaN          NaN   
11          t       NaN        NaN  NaN       NaN  unmeasured   

In [9]:
from actionrules.actionRulesDiscovery import ActionRulesDiscovery
actionRulesDiscovery = ActionRulesDiscovery()
actionRulesDiscovery.load_pandas(frame)
actionRulesDiscovery.fit_classification_rules(stable_antecedents = cols[0:6],
                                 flexible_antecedents = cols[6:],
                                 consequent = 'target',
                                 conf_col = 'confidence',
                                 supp_col = 'support',
                                 desired_classes = [1])

In [10]:
len(actionRulesDiscovery.get_action_rules())

6

In [11]:
for rule in actionRulesDiscovery.get_action_rules_representation():
    print(rule)
    print(" ")

r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: absent → elevated) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: absent → ?) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: elevated → absent) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: elevated → ?) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: ? → absent) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
r = [(age-gt-60: t) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: ? → elevated) ] ⇒ [target: 0 → 1] with support: 0.006006006006006006 and confidence: 0.6666666666666666
 
